In [36]:
import os, re, shutil, random, csv
from pathlib import Path
from collections import defaultdict, Counter

SRC_ROOT   = Path("C:/Users/Miles/Desktop/rock and mineral type/rocks_three")        
OUT_BASE   = Path("C:/Users/Miles/Desktop/rock and mineral type/processed_datasets") # both libraries will be created under here
SPLIT      = (0.75, 0.15, 0.10)         # train/val/test
SEED       = 42
IMG_EXT    = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}
COPY_FILES = True  # True=copy; False=symlink when possible to save disk
MIN_MINERAL_COUNT = 5


In [20]:
random.seed(SEED)

def clean_mineral_from_filename(name: str) -> str:
    """
    Parse mineral/rock name from filename.
    Examples:
      '100Andesite.png'           -> 'andesite'
      '101Quartz_diorite.png'     -> 'quartz_diorite'
      '12-Blue-Schist.JPG'        -> 'blue_schist'
    """
    stem = Path(name).stem
    stem = stem.replace("-", "_").replace(" ", "_")
    stem = re.sub(r"^\d+_?", "", stem)       # remove leading digits + optional underscore
    stem = re.sub(r"__+", "_", stem)         # collapse multiple underscores
    stem = re.sub(r"[^A-Za-z_]", "", stem)   # keep letters/underscores
    stem = stem.strip("_")
    return stem.lower()


In [21]:
def list_images(src_root: Path):
    records = []  # list of dicts with keys: path, rock_type, mineral
    for rock_type_dir in ["igneous", "metamorphic", "sedimentary"]:
        rdir = src_root / rock_type_dir
        if not rdir.exists():
            print(f"Warning: missing folder {rdir}")
            continue
        for p in rdir.rglob("*"):
            if p.suffix.lower() in IMG_EXT and p.is_file():
                mineral = clean_mineral_from_filename(p.name)
                if not mineral:
                    mineral = "unknown"
                records.append({"path": p.resolve(), "rock_type": rock_type_dir, "mineral": mineral})
    return records

In [22]:
def stratified_split(items, key_fn, split=SPLIT):
    by_key = defaultdict(list)
    for it in items:
        by_key[key_fn(it)].append(it)
    train_p, val_p, test_p = split
    out = {"train": [], "val": [], "test": []}
    for k, lst in by_key.items():
        lst = lst.copy()
        random.shuffle(lst)
        n = len(lst)
        n_train = int(round(n * train_p))
        n_val   = int(round(n * val_p))
        if n_train + n_val > n:
            n_val = max(0, n - n_train)
        n_test  = max(0, n - n_train - n_val)
        out["train"] += lst[:n_train]
        out["val"]   += lst[n_train:n_train+n_val]
        out["test"]  += lst[n_train+n_val:]
    return out


In [23]:
def safe_mkdir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

In [24]:
def place_files(splits, dest_root: Path, class_key: str):
    """
    class_key ∈ {'rock_type', 'mineral'}
    writes under dest_root/images/{train,val,test}/{class}/filename
    """
    for stage, rows in splits.items():
        for r in rows:
            cls = r[class_key]
            src = r["path"]
            dst_dir = dest_root / "images" / stage / cls
            safe_mkdir(dst_dir)
            dst = dst_dir / src.name
            if dst.exists():
                continue
            if COPY_FILES:
                shutil.copy2(src, dst)
            else:
                try:
                    rel = os.path.relpath(src, dst.parent)
                    dst.symlink_to(rel)
                except Exception:
                    shutil.copy2(src, dst)

In [25]:
def write_classes_txt(dest_root: Path, classes):
    with open(dest_root / "classes.txt", "w", encoding="utf-8") as f:
        for c in classes:
            f.write(c + "\n")

In [26]:
def write_manifest_csv(dest_root: Path, rows, extra_cols=("rock_type","mineral")):
    out_csv = dest_root / "manifest.csv"
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["stage", "class", "filename", *extra_cols])
        for stage, lst in rows.items():
            for r in lst:
                w.writerow([stage, r["rock_type" if "rock_type" in extra_cols else "mineral"], Path(r["path"]).name, r["rock_type"], r["mineral"]])
    return out_csv

In [35]:
def count_split(root: Path):
    def nimgs(d):
        return sum(1 for p in d.iterdir() if p.is_file() and p.suffix.lower() in IMG_EXT)
    stats = {}
    for stage in ("train","val","test"):
        stage_dir = root/"images"/stage
        total = 0
        if stage_dir.exists():
            for cls in sorted([d for d in stage_dir.iterdir() if d.is_dir()]):
                k = (stage, cls.name)
                stats[k] = nimgs(cls); total += stats[k]
        stats[(stage,"_total_")] = total
    return stats

In [27]:
# 1) Scan source
all_items = list_images(SRC_ROOT)
if not all_items:
    raise RuntimeError("No images found. Check SRC_ROOT/config.")

print(f"Found {len(all_items)} images")
print("Sample:", all_items[0])

Found 10830 images
Sample: {'path': WindowsPath('C:/Users/Miles/Desktop/rock and mineral type/rocks_three/igneous/100Andesite.png'), 'rock_type': 'igneous', 'mineral': 'andesite'}


In [28]:
# 2) Build RockType dataset (3-way)
rock_classes = sorted({r["rock_type"] for r in all_items})
rt_splits = stratified_split(all_items, key_fn=lambda r: r["rock_type"])
RT_ROOT = OUT_BASE / "RockType"
place_files(rt_splits, RT_ROOT, class_key="rock_type")
write_classes_txt(RT_ROOT, rock_classes)
rt_csv = write_manifest_csv(RT_ROOT, rt_splits, extra_cols=("rock_type","mineral"))
print(f"[RockType] classes={rock_classes}  -> {RT_ROOT}")
print(f"Manifest: {rt_csv}")

[RockType] classes=['igneous', 'metamorphic', 'sedimentary']  -> C:\Users\Miles\Desktop\rock and mineral type\processed_datasets\RockType
Manifest: C:\Users\Miles\Desktop\rock and mineral type\processed_datasets\RockType\manifest.csv


In [29]:
# 3) Build Mineral dataset (N-way)
mineral_counts = Counter(r["mineral"] for r in all_items)
keep_set = {m for m, c in mineral_counts.items() if c >= MIN_MINERAL_COUNT}
filtered = [r for r in all_items if r["mineral"] in keep_set]

print(f"Mineral classes (≥{MIN_MINERAL_COUNT} imgs): {len(keep_set)} kept / {len(mineral_counts)} total")

mn_splits = stratified_split(filtered, key_fn=lambda r: r["mineral"])
MN_ROOT = OUT_BASE / "Mineral"
mineral_classes = sorted(keep_set)
place_files(mn_splits, MN_ROOT, class_key="mineral")
write_classes_txt(MN_ROOT, mineral_classes)
mn_csv = write_manifest_csv(MN_ROOT, mn_splits, extra_cols=("rock_type","mineral"))
print(f"[Mineral] classes={len(mineral_classes)}  -> {MN_ROOT}")
print(f"Manifest: {mn_csv}")


Mineral classes (≥5 imgs): 51 kept / 51 total
[Mineral] classes=51  -> C:\Users\Miles\Desktop\rock and mineral type\processed_datasets\Mineral
Manifest: C:\Users\Miles\Desktop\rock and mineral type\processed_datasets\Mineral\manifest.csv


In [30]:
# 4) Quick summary
def count_tree(root: Path):
    out = defaultdict(int)
    total = 0
    for stage in ("train","val","test"):
        for clsdir in (root/"images"/stage).glob("*"):
            if not clsdir.is_dir(): 
                continue
            n = len([p for p in clsdir.iterdir() if p.suffix.lower() in IMG_EXT])
            out[(stage, clsdir.name)] = n
            total += n
    return out, total

for name, root in [("RockType", RT_ROOT), ("Mineral", MN_ROOT)]:
    counts, total = count_tree(root)
    print(f"\n{name} dataset @ {root}")
    for stage in ("train","val","test"):
        stage_total = sum(v for (s,_), v in counts.items() if s==stage)
        print(f" {stage}: {stage_total}")
        # show top few classes
        sample = [(c, n) for (s,c), n in counts.items() if s==stage]
        print("   ", sorted(sample)[:5], "...")
    print(f" TOTAL: {total}")


RockType dataset @ C:\Users\Miles\Desktop\rock and mineral type\processed_datasets\RockType
 train: 8122
    [('igneous', 2886), ('metamorphic', 2888), ('sedimentary', 2348)] ...
 val: 1625
    [('igneous', 577), ('metamorphic', 578), ('sedimentary', 470)] ...
 test: 1083
    [('igneous', 385), ('metamorphic', 385), ('sedimentary', 313)] ...
 TOTAL: 10830

Mineral dataset @ C:\Users\Miles\Desktop\rock and mineral type\processed_datasets\Mineral
 train: 8118
    [('amphibolite', 223), ('andesite', 220), ('anthracite', 163), ('basalt', 236), ('blueschist', 191)] ...
 val: 1623
    [('amphibolite', 45), ('andesite', 44), ('anthracite', 33), ('basalt', 47), ('blueschist', 38)] ...
 test: 1089
    [('amphibolite', 29), ('andesite', 29), ('anthracite', 21), ('basalt', 32), ('blueschist', 26)] ...
 TOTAL: 10830


In [31]:
from ultralytics import YOLO
from pathlib import Path
import torch, os
print("Torch:", torch.__version__, "CUDA:", torch.cuda.is_available())

Torch: 2.9.0+cpu CUDA: False


In [32]:
#Paths

In [37]:
from pathlib import Path
ROCKTYPE_DATA = Path("C:/Users/Miles/Desktop/rock and mineral type/processed_datasets/RockType")
MINERAL_DATA  = Path("C:/Users/Miles/Desktop/rock and mineral type/processed_datasets/Mineral")

assert (ROCKTYPE_DATA/"images/train").exists(), "RockType dataset missing train split"
assert (MINERAL_DATA/"images/train").exists(),  "Mineral dataset missing train split"


In [ ]:
#Train RockType (3-class) with YOLOv8-cls

In [38]:
rocktype_model = YOLO("yolov8n-cls.pt")  # or "yolov8s-cls.pt" for more capacity

rocktype_results = rocktype_model.train(
    data=str(ROCKTYPE_DATA/"images"),  # must contain train/, val/, (optional test/)
    epochs=30,
    imgsz=224,
    lr0=1e-3,
    batch=32,
    patience=10,        # early stop
    project="runs_cls",
    name="rocktype_y8n",
    verbose=True
)


Ultralytics 8.3.227  Python-3.11.5 torch-2.9.0+cpu CPU (AMD Ryzen 9 5900HX with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\Miles\Desktop\rock and mineral type\processed_datasets\RockType\images, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=rocktype_y8n, nbs=64, nms=False, opset=None, optimize=False, optimize

In [ ]:
#Evaluate on the test split

In [39]:
rocktype_model.val(
    data=str(ROCKTYPE_DATA/"images"),
    split="test",
    imgsz=224,
    batch=64
)


Ultralytics 8.3.227  Python-3.11.5 torch-2.9.0+cpu CPU (AMD Ryzen 9 5900HX with Radeon Graphics)
YOLOv8n-cls summary (fused): 30 layers, 1,438,723 parameters, 0 gradients, 3.3 GFLOPs
train: C:\Users\Miles\Desktop\rock and mineral type\processed_datasets\RockType\images\train... found 8122 images in 3 classes  
val: C:\Users\Miles\Desktop\rock and mineral type\processed_datasets\RockType\images\val... found 1625 images in 3 classes  
test: C:\Users\Miles\Desktop\rock and mineral type\processed_datasets\RockType\images\test... found 1083 images in 3 classes  
test: Fast image access  (ping: 0.10.0 ms, read: 2.10.4 MB/s, size: 14.2 KB)
test: Scanning C:\Users\Miles\Desktop\rock and mineral type\processed_datasets\RockType\images\test... 1083 images, 0 corrupt: 100% ━━━━━━━━━━━━ 1083/1083 821.8it/s 1.3s0.1s
test: New cache created: C:\Users\Miles\Desktop\rock and mineral type\processed_datasets\RockType\images\test.cache
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 17/17

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000024E5FAA17D0>
curves: []
curves_results: []
fitness: 0.8079409003257751
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.6158818006515503, 'metrics/accuracy_top5': 1.0, 'fitness': 0.8079409003257751}
save_dir: WindowsPath('C:/Users/Miles/runs/classify/val')
speed: {'preprocess': 0.00045032316555689575, 'inference': 3.7879076639503335, 'loss': 2.0867930157651862e-05, 'postprocess': 5.484760621956456e-05}
task: 'classify'
top1: 0.6158818006515503
top5: 1.0

In [ ]:
sum(1 for _ in (ROCKTYPE_DATA/"images/test").rglob("*.png"))


In [ ]:
from pathlib import Path
#recur call
IMG_EXT = {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}
test_dir = ROCKTYPE_DATA/"images"/"test"
test_imgs = [str(p) for p in test_dir.rglob("*") if p.suffix.lower() in IMG_EXT]

print(f"{len(test_imgs)} test images")
pred = rocktype_model.predict(
    source=test_imgs,   # list of file paths
    imgsz=224,
    batch=64,
    save=True           # outputs under runs/classify/predict*
)


In [ ]:
#Train Mineral (N-class) with YOLOv8-cls

In [ ]:
mineral_model = YOLO("yolov8s-cls.pt")  # n=smaller, s=balanced, m/l if you have the GPU budget

mineral_results = mineral_model.train(
    data=str(MINERAL_DATA/"images"),
    epochs=50,
    imgsz=224,
    lr0=1e-3,
    batch=32,
    patience=12,
    project="runs_cls",
    name="mineral_y8s"
)
mineral_model.val(
    data=str(MINERAL_DATA/"images"),
    split="test",
    imgsz=224,
    batch=64
)


Ultralytics 8.3.227  Python-3.11.5 torch-2.9.0+cpu CPU (AMD Ryzen 9 5900HX with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\Miles\Desktop\rock and mineral type\processed_datasets\Mineral\images, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=mineral_y8s, nbs=64, nms=False, opset=None, optimize=False, optimizer=

In [ ]:
#Augmentations: Ultralytics applies light augs by default; 
#you can increase with augment=True and flipud/fliplr/degrees in overrides.

In [ ]:
mineral_model.train(
    data=str(MINERAL_DATA/"images"),
    epochs=60, imgsz=224, batch=32,
    lr0=8e-4, patience=15,
    augment=True, fliplr=0.5, degrees=15, scale=0.2
)


In [ ]:
#Export for deployment (TorchScript/ONNX)

In [ ]:
# RockType
rocktype_model.export(format="torchscript")  # creates rocktype_y8n.torchscript
rocktype_model.export(format="onnx", opset=12)

# Mineral
mineral_model.export(format="torchscript")
mineral_model.export(format="onnx", opset=12)


In [ ]:
# Confusion matrix + per-class report for YOLOv8 classification
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, top_k_accuracy_score
import csv

IMG_EXT = {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}

def eval_cls_dataset(model, dataset_root: Path, split="test", imgsz=224, batch=64, save_dir=None, topk=(1,5)):
    """
    model: a loaded/trained ultralytics.YOLO classification model
    dataset_root: path like .../processed_datasets/RockType  (contains images/{train,val,test}/<class>/*)
    split: which split to evaluate ("test" recommended)
    """
    root = Path(dataset_root)
    split_dir = root/"images"/split
    assert split_dir.exists(), f"Missing split folder: {split_dir}"

    # 1) Collect files and ground-truth labels from folder names
    class_names = sorted([d.name for d in split_dir.iterdir() if d.is_dir()])
    name_to_id = {n:i for i,n in enumerate(class_names)}  # GT mapping from folder order
    files, y_true = [], []
    for cls in class_names:
        for p in (split_dir/cls).rglob("*"):
            if p.suffix.lower() in IMG_EXT and p.is_file():
                files.append(str(p))
                y_true.append(name_to_id[cls])
    assert len(files) > 0, f"No images found under {split_dir}."

    # 2) Predict
    res = model.predict(source=files, imgsz=imgsz, batch=batch, verbose=False)
    # top-1 predictions + probs
    y_pred = [int(r.probs.top1) for r in res]
    top1conf = [float(r.probs.top1conf) for r in res]
    # top-k arrays (for k in topk)
    # r.probs.top5 gives indices for k=5 if available; fall back safely
    def get_topk_indices(r, k):
        if hasattr(r.probs, "top5"):
            topk_idx = r.probs.top5[:k]
        else:
            # general case: argsort of full probs (descending)
            topk_idx = np.argsort(-r.probs.data.cpu().numpy())[:k]
        return list(map(int, topk_idx))
    y_topk = {k: [get_topk_indices(r, k) for r in res] for k in topk}

    # 3) Metrics
    acc = accuracy_score(y_true, y_pred)
    topk_acc = {k: np.mean([yt in cand for yt, cand in zip(y_true, y_topk[k])]) for k in topk}
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(class_names))))

    # 4) Report dict (sklearn) -> CSV
    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True, zero_division=0)

    # 5) Save assets
    if save_dir is None:
        save_dir = Path("runs") / "classify_reports" / (root.name + f"_{split}")
    save_dir.mkdir(parents=True, exist_ok=True)

    # Confusion matrix figure (counts and normalized)
    def plot_cm(cm, labels, normalize=False, fname="confusion_matrix.png"):
        M = cm.astype(np.float32)
        if normalize:
            row_sums = M.sum(axis=1, keepdims=True) + 1e-12
            M = M / row_sums
        fig, ax = plt.subplots(figsize=(max(6, 0.8*len(labels)), max(5, 0.8*len(labels))))
        im = ax.imshow(M, interpolation='nearest')
        ax.figure.colorbar(im, ax=ax)
        ax.set(xticks=np.arange(len(labels)), yticks=np.arange(len(labels)),
               xticklabels=labels, yticklabels=labels,
               ylabel='True label', xlabel='Predicted label',
               title=('Normalized ' if normalize else '') + 'Confusion Matrix')
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
        # annotate a few cells (avoid clutter for huge matrices)
        if len(labels) <= 20:
            thresh = M.max() / 2.
            for i in range(M.shape[0]):
                for j in range(M.shape[1]):
                    txt = f"{M[i, j]:.2f}" if normalize else f"{int(cm[i, j])}"
                    ax.text(j, i, txt, ha="center", va="center",
                            color="white" if M[i, j] > thresh else "black", fontsize=8)
        fig.tight_layout()
        fig.savefig(save_dir/fname, dpi=180, bbox_inches="tight")
        plt.show()

    plot_cm(cm, class_names, normalize=False, fname="confusion_matrix_counts.png")
    plot_cm(cm, class_names, normalize=True,  fname="confusion_matrix_normalized.png")

    # 6) Per-class bar chart (F1-score)
    f1s = [report[c]["f1-score"] for c in class_names]
    fig, ax = plt.subplots(figsize=(max(6, 0.5*len(class_names)), 4))
    ax.bar(range(len(class_names)), f1s)
    ax.set_xticks(range(len(class_names)))
    ax.set_xticklabels(class_names, rotation=45, ha='right')
    ax.set_ylabel("F1-score")
    ax.set_title("Per-class F1")
    fig.tight_layout()
    fig.savefig(save_dir/"per_class_f1.png", dpi=180, bbox_inches="tight")
    plt.show()

    # 7) Write CSV with full report
    with open(save_dir/"classification_report.csv","w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        header = ["class","precision","recall","f1_score","support"]
        writer.writerow(header)
        for c in class_names:
            row = report[c]
            writer.writerow([c, row["precision"], row["recall"], row["f1-score"], int(row["support"])])
        # overall rows
        writer.writerow([])
        for k in ["accuracy","macro avg","weighted avg"]:
            if k=="accuracy":
                writer.writerow(["accuracy", acc, "", "", sum(report[c]["support"] for c in class_names)])
            else:
                row = report[k]
                writer.writerow([k, row["precision"], row["recall"], row["f1-score"], int(row["support"])])

    # 8) Log a quick summary
    print(f"Saved reports to: {save_dir}")
    print(f"Top-1 acc: {acc:.4f}", " | ", "  ".join([f"top-{k}: {topk_acc[k]:.4f}" for k in topk]))
    return {"acc": acc, "topk": topk_acc, "report": report, "cm": cm, "classes": class_names, "save_dir": save_dir}


In [ ]:
RockType = eval_cls_dataset(
    model=rocktype_model,
    dataset_root=ROCKTYPE_DATA,   # Path("processed_datasets/RockType")
    split="test",                 # or "val"
    imgsz=224,
    batch=64
)


In [ ]:
MineralCall= eval_cls_dataset(
    model=mineral_model,
    dataset_root=MINERAL_DATA,    # Path("processed_datasets/Mineral")
    split="test",
    imgsz=224,
    batch=64
)


In [ ]:
#Using both models together (prototype decision layer)

In [ ]:
from ultralytics.utils.torch_utils import select_device
device = select_device(0 if torch.cuda.is_available() else "cpu")

def classify_rock(image_path):
    # 1) rock type
    rt = rocktype_model.predict(source=image_path, imgsz=224, device=device, verbose=False)[0]
    rt_name = rt.names[int(rt.probs.top1)]
    rt_conf = float(rt.probs.top1conf)

    # 2) mineral
    mn = mineral_model.predict(source=image_path, imgsz=224, device=device, verbose=False)[0]
    mn_name = mn.names[int(mn.probs.top1)]
    mn_conf = float(mn.probs.top1conf)

    return {"rock_type": (rt_name, rt_conf), "mineral": (mn_name, mn_conf)}

print(classify_rock(next((ROCKTYPE_DATA/"images/test/igneous").glob("*.png"))))
